# 18. `with`, `match`, and `else` Blocks

## Context Managers and `with` Blocks

Context manager objects exist to control a `with` statement, just like iterators exist to control a `for` statement.

The `with` statement was designed to simplify some common uses of `try`/`finally`, which guarantees that some operation is performed after a block of code, even if the block is terminated by `return`, an exception, or a `sys.exit()` call. The code in the `finally` clause usually releases a critical resource or restores some previous state that was temporally changed.

In [3]:
with open('totalsum.py') as fp:
    src = fp.read(60)

len(src)

60

In [4]:
fp

<_io.TextIOWrapper name='totalsum.py' mode='r' encoding='UTF-8'>

In [5]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [7]:
try:
    print(f"{fp.read(60)=}")
except Exception as e:
    print(f"{e=}")

e=ValueError('I/O operation on closed file.')


In [8]:
import sys

class LookingGlass:

    def __enter__(self):
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

In [9]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [10]:
what

'JABBERWOCKY'

In [11]:
print('Back to normal.')

Back to normal.


In [12]:
manager = LookingGlass()
manager

In [13]:
monster = manager.__enter__()

In [14]:
monster == 'JABBERWOCKY'

True

In [15]:
manager

In [16]:
manager.__exit__(None, None, None)

In [17]:
monster

'JABBERWOCKY'

## The `contextlib` Utilities

Before rolling your own context manager classess, take a look at `contextlib`.

The `contextlib` module provides classes and a decorator that are more widely applicable than the decorators just mentioned:

`@contextmanager`

A decorator that lets you build a context manager from a simple generator function.

`AbstractContextManager`

An ABC that formalizes the context manager interface, and makes it a bit easier to create context manager classes by subclassing.

`ContextDecorator`

A base class for defining class-based context manager that can also be used as function decorators, running the entire function with a managed context.

`ExitStack`

A context manager that lets you enter a variable number of context managers. When the `with` block ends, `ExitStack` calls the stacked context managers' `__exit__` methods in LIFO order.

### Using the `@contextmanager`

The `@contextmanager` decorator is an elegant and practical tool that brings together three distinctive Python features: a function decorator, a generator, and the `with` statement.

In [18]:
import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write

In [19]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [20]:
what

'JABBERWOCKY'

In [21]:
print('back to normal')

back to normal


In [22]:
import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

In [23]:
@looking_glass()
def verse():
    print('The time has come')

In [24]:
verse()

emoc sah emit ehT


In [25]:
print('back to normal')

back to normal


**Scheme** code example

```scheme
(define (mod m n)
    (- m (* n (quotient m n ))))

(define (gcd m n)
    (if (= n 0)
        m
        (gcd n (mod m n))))

(display (gcd 18 45))
```

## Do This, Then That: `else` Blocks Beyond `if`

This is no secret, but it is an underappreciated language feature: the `else` clause can be used not only in `if` statements but also in `for`, `while` and `try` statements.

Here are the rules:

`for`

The `else` block will run only if and when the `for` loop runs to completion,

`while` 

The `else` block will run only if and when the `while` loop exits because the condition become _falsy_.

`try`

The `else` block will run only if no exception is raised in the `try` block. The the official docs alse state: "Exceptions in the `else` clause are not handled by the preceding `except` clauses."